In [20]:
import pandas as pd, numpy as np, re

ATT_PATH = "Attendance.xlsx"

# Load and normalize
att = pd.concat(pd.read_excel(ATT_PATH, sheet_name=None, dtype=object).values(), ignore_index=True)
def norm(s): s=str(s).strip().lower(); s=re.sub(r"[^\w]+","_",s); return re.sub(r"_+","_",s).strip("_")
att.columns = [norm(c) for c in att.columns]

# Key + year
key = next((c for c in att.columns if "student" in c and "key" in c), None)
if not key: raise ValueError("student key column not found")
att = att.rename(columns={key:"mask_studentpersonkey"})
att["mask_studentpersonkey"] = att["mask_studentpersonkey"].astype(str).str.strip()

year_col = next((c for c in ["schoolyearid","school_year_id","school_year","schoolyear","year"] if c in att.columns), None)

# Pick fields
days_col = next((c for c in ["school_days_enrolled","days_enrolled","membership_days","days_possible","total_days","days"] if c in att.columns), None)
abs_col  = next((c for c in ["sum_of_absences","total_absences","absences","days_absent","absent_days"] if c in att.columns), None)
pct_col  = next((c for c in ["percent_present","attendance_rate","attendance_percent","present_percent"] if c in att.columns), None)
tardy_col= next((c for c in ["sum_of_tardies","tardies","tardy_count"] if c in att.columns), None)

def numify(s): return pd.to_numeric(pd.Series(s).astype(str).str.replace(",","",regex=False), errors="coerce")
days   = numify(att[days_col]) if days_col else 0
absent = numify(att[abs_col])  if abs_col  else np.nan
pct    = numify(att[pct_col])  if pct_col  else np.nan
pct    = np.where(pd.isna(pct), np.nan, np.where(pct>1, pct/100.0, pct))

present = np.where(~pd.isna(pct), (pct*days).round(0), days - absent)
present = pd.Series(present).clip(lower=0).fillna(0)
absent  = pd.Series(absent).fillna(days - present).clip(lower=0)
days    = pd.Series(days).fillna(0)
days    = np.where((days==0) & ((present>0)|(absent>0)), present+absent, days)

base = pd.DataFrame({
    "mask_studentpersonkey": att["mask_studentpersonkey"],
    "year": att[year_col] if year_col else np.nan,
    "present": present,
    "absent": absent,
    "days": days,
})
if tardy_col:
    base["tardy"] = numify(att[tardy_col]).fillna(0)

# Totals by student
tot = (base.groupby("mask_studentpersonkey", as_index=False)
           .agg(total_present=("present","sum"),
                total_absent=("absent","sum"),
                total_days=("days","sum"),
                total_tardy=("tardy","sum") if "tardy" in base else ("present","sum")))
if "total_tardy" not in tot: tot = tot.drop(columns=["total_tardy"], errors="ignore")
tot["present_pct"] = (tot["total_present"]/tot["total_days"]).replace([np.inf,np.nan],0)
tot["chronic_absent_10pct"] = (tot["total_absent"] >= 0.10*tot["total_days"]).astype(int)
tot = tot.sort_values("mask_studentpersonkey")
tot.to_csv("attendance_totals_by_student.csv", index=False)

# Wide by student-year (only if year exists)
if year_col:
    by = (base.dropna(subset=["year"])
              .groupby(["mask_studentpersonkey","year"], as_index=False)
              .agg(present=("present","sum"),
                   absent=("absent","sum"),
                   days=("days","sum"),
                   tardy=("tardy","sum") if "tardy" in base else ("present","sum")))
    by["present_pct"] = (by["present"]/by["days"]).replace([np.inf,np.nan],0)
    by["chronic_absent_10pct"] = (by["absent"] >= 0.10*by["days"]).astype(int)

    wide_parts = []
    for col in ["present","absent","tardy","days","present_pct","chronic_absent_10pct"]:
        if col not in by.columns: continue
        w = by.pivot(index="mask_studentpersonkey", columns="year", values=col)
        w.columns = [f"{col}_{int(c)}" for c in w.columns]
        wide_parts.append(w)

    wide = pd.concat(wide_parts, axis=1).reset_index().sort_values("mask_studentpersonkey")
    wide.to_csv("attendance_totals_by_student_year.csv", index=False)

# Output Summary
print("attendance_totals_by_student.csv:", tot.shape)
print("present / absent / days sums:",
      int(tot.total_present.sum()),
      int(tot.total_absent.sum()),
      int(tot.total_days.sum()))
if year_col:
    print("attendance_totals_by_student_year.csv:", wide.shape)


attendance_totals_by_student.csv: (85809, 7)
present / absent / days sums: 31858206 1748150 33606341
attendance_totals_by_student_year.csv: (85809, 25)
